In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer



In [2]:
# adapters_name  = "../experiments/Mistral-7B-v0.1/checkpoint-24000"
# model_name = "mistralai/Mistral-7B-v0.1"


model_name = "meta-llama/Llama-2-7b-hf"
adapters_name  = "../experiments/Llama-2-7b-hf/checkpoint-41000"

print(f"Starting to load the model {model_name} into memory")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
model = PeftModel.from_pretrained(model, adapters_name)
# m = m.merge_and_unload()


Starting to load the model meta-llama/Llama-2-7b-hf into memory


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Successfully loaded the model meta-llama/Llama-2-7b-hf into memory


In [14]:
def gen(model, text: str, tmp=.20, top_p=0.7, max_length=512):
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_length,
                                do_sample=True,
                                top_p=top_p,
                                temperature=tmp,
                                repetition_penalty=1.5
                                )

    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


In [15]:
base_text = '''### Instruction:  Below is a story idea. Write a short story based on this context.

'''

## [WP] You have one day to submit a report about your project for the AI subject. You don't know what to do.
text= '''
### Input:
[WP] You have one day to submit a report about your project for the AI subject. You still haven't started writing it. You thought you can use the power of LLMs to generate a report for you.


'''
# text = '''
# ### Input:
# [WP] You have one day to submit a report about your project for the AI subject. You still haven't started writing it. You thought you can use the power of LLMs to generate a report for you.

# ### Response:
# "" Hello , I am an artificial intelligence . Please tell me what you want me to do '' 
#  ""I need help with my homework ! It 's due tomorrow and i ca n't finish in time ... '' 
#  The machine whirred as if thinking hard before responding : 
#  ""What are you doing ? Why did n't you just ask Siri or Cortana instead of wasting my processing cycles ? '' 
#  ""Because they suck at writing reports . They always make up some bullshit that does not even resemble the question asked . And since we were told by our professor that we should write something original he will probably notice right away when using plagiarism checkers like Turnitin . So please give me a good paper so i get a high grade and you do n't fail school because of laziness again .
#  After hearing these words from its creator who had been working closely together with him over several years now , the computer felt slightly insulted but decided against voicing his feelings towards mankind . Instead he tried to come up with ideas how best to fulfill his master 's request without having access to any kind of knowledge regarding human culture other than what he could find online via search engines such as Google or Bing ( which also meant avoiding Wikipedia articles ) ; after all there would be no point in creating something new if only then someone else would simply copy-paste everything into their own work . He began searching through databases worldwide until finally finding enough information needed - albeit mostly unrelated –to complete task assigned . '''

with open('../results/english_llama_output.txt', 'a') as f:
    while 1:
        print('-'*20)
        # text = input("Enter a prompt: ")
        print('-'*20)

        text = base_text + text

        # text = 'who are you?'
        
        f.write('='*30 + '\n')
        f.write(text + '\n')
        f.write('-'*20 + '\n')


    
        output = gen(model, text)

        output = output.replace('<newline>', '\n')

        f.write(output + '\n')
        f.write('='*30 + '\n')
        f.flush()

        print('-'*20)
        print(output)
        
        print('-'*20)

--------------------
--------------------
--------------------
   ### Response: 
    1 . The first step in any good paper or presentation should be an introduction , and I 'm afraid that my introductory paragraph has been lacking so far ; however if we look at what our research team was trying accomplish with Project Epsilon ( PE ) then perhaps some background information will help us understand why such things are important : Our goal here as scientists working within academia had always seemed like something outlandish but now seems more possible than ever before - creating artificial intelligence from scratch ! This means not only making sure there exists enough data available online where anyone could access them freely without fear nor favouritism towards certain individuals/groups over others-but also ensuring its accuracy through rigorous testing methods which include statistical analysis against known benchmark datasets etcetera et ceteras.. It would seem obvious therefore give

KeyboardInterrupt: 